# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [2]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [3]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [4]:
bosonNI+bosonI

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [5]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [6]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [7]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [8]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    return lam*qs[n]*qs[n]*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 - q_{0}*q_{1} - 2*q_{0}*q_{1}**3 - q_{0}*q_{2} + 2*q_{0}*q_{2}**3 + 14.0*q_{0}**2 - 24*q_{0}**2*\chi^{\dagger}_{0}*\chi_{0} + 2*q_{0}**3*q_{1} - 2*q_{0}**3*q_{2} + 4*q_{0}**6 - q_{1}*q_{0} + 2*q_{1}*q_{0}**3 - q_{1}*q_{2} - 2*q_{1}*q_{2}**3 - 10.0*q_{1}**2 + 24*q_{1}**2*\chi^{\dagger}_{1}*\chi_{1} - 2*q_{1}**3*q_{0} + 2*q_{1}**3*q_{2} + 4*q_{1}**6 - q_{2}*q_{0} - 2*q_{2}*q_{0}**3 - q_{2}*q_{1} + 2*q_{2}*q_{1}**3 + 14.0*q_{2}**2 - 24*q_{2}**2*\chi^{\dagger}_{2}*\chi_{2} + 2*q_{2}**3*q_{0} - 2*q_{2}**3*q_{1} + 4*q_{2}**6)/8

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

\chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{1}*a_{1}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{1}*a_{1}**2/16 - a^{\dagger}_{0}*a^{\dagger}_{1}**2*a_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{1}**3/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + a^{\dagger}_{0}*a^{\dagger}_{2}*a_{2}*a^{\dagger}_{2}/16 + a^{\dagger}_{0}*a^{\dagger}_{2}*a_{2}**2/16 + a^{\dagger}_{0}*a^{\dagger}_{2}**2*a_{2}/16 + a^{\dagger}_{0}*a^{\dagger}_{2}**3/16 + 9*a^{\dagger}_{0}*a_{0}/8 - 3*a^{\dagger}_{0}*a_{0}*\chi^{\dagger}_{0}*\chi_{0}/2 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a^{\dagger}_{2}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2/16 + a^{\dagger}

In [10]:
tstTerm=aops[3]**6/16
mat=convert_term_to_matrix(tstTerm,8,N,aops,adags,xs,xdags)
mat

<4096x4096 sparse matrix of type '<class 'numpy.object_'>'
	with 1024 stored elements in Compressed Sparse Row format>

In [11]:
print(mat)

  (0, 12)	1.67705094814301
  (1, 13)	1.67705094814301
  (2, 14)	4.43705987930298
  (3, 15)	4.43705987930298
  (16, 28)	1.67705094814301
  (17, 29)	1.67705094814301
  (18, 30)	4.43705987930298
  (19, 31)	4.43705987930298
  (32, 44)	1.67705094814301
  (33, 45)	1.67705094814301
  (34, 46)	4.43705987930298
  (35, 47)	4.43705987930298
  (48, 60)	1.67705094814301
  (49, 61)	1.67705094814301
  (50, 62)	4.43705987930298
  (51, 63)	4.43705987930298
  (64, 76)	1.67705094814301
  (65, 77)	1.67705094814301
  (66, 78)	4.43705987930298
  (67, 79)	4.43705987930298
  (80, 92)	1.67705094814301
  (81, 93)	1.67705094814301
  (82, 94)	4.43705987930298
  (83, 95)	4.43705987930298
  (96, 108)	1.67705094814301
  :	:
  (3987, 3999)	4.43705987930298
  (4000, 4012)	1.67705094814301
  (4001, 4013)	1.67705094814301
  (4002, 4014)	4.43705987930298
  (4003, 4015)	4.43705987930298
  (4016, 4028)	1.67705094814301
  (4017, 4029)	1.67705094814301
  (4018, 4030)	4.43705987930298
  (4019, 4031)	4.43705987930298
  (4032, 

/home/chris/anaconda3/envs/QuantumComputing/lib/python3.9/site-packages/scipy/sparse/_sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


In [13]:
for cutoff in [2,3,4,5,6,7,8,9,10]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 5.0743 seconds
evals timer took: 0.0027 seconds
Lambda =  2
[ 0.0500414 +2.4659816e-10j  0.05004131+2.2609234e-09j
  0.05004135-4.4795953e-10j  0.05004134-2.7546445e-09j
 -0.4499581 +1.2257945e-08j  0.1874998 -6.1010230e-10j
  0.18749993-1.9037238e-09j -0.4499581 -7.2593931e-09j
  0.05004137+4.7447806e-09j  0.05004138-5.9703336e-09j]

toHam timer took: 7.7552 seconds
evals timer took: 0.0046 seconds
Lambda =  3
[0.04553825+7.5153384e-11j 0.05645379-2.9590326e-09j
 0.17575875-6.0867400e-09j 0.18965445-1.4369272e-08j
 0.18291554-2.6024152e-09j 0.23131132+1.4023744e-09j
 0.42347932-7.5190944e-11j 0.4187012 +1.2954183e-08j
 0.5500074 -6.8686845e-08j 0.57022   -6.0111880e-09j]

toHam timer took: 15.4716 seconds
evals timer took: 0.0082 seconds
Lambda =  4
[0.16245374+2.7889784e-09j 0.631823  +1.9211269e-08j
 0.8892409 +5.1052118e-09j 1.0975862 -3.6012185e-08j
 1.0958065 -1.7540440e-08j 1.3383257 -1.6394901e-08j
 1.3511075 -1.7606581e-08j 1.5561857 -9.3870902e-09j
 1.565022

KeyboardInterrupt: 